In [1]:
from crossword_solver.crossword_detection import detect_crossword_from_file
from crossword_solver.solver_helpers import find_whole_crossword_candidates, reorder_crossword_hints, solving_algorithm
from crossword_solver.candidate_search_helpers import lemmatise_text

from copy import deepcopy
import re


Resource 'estwordnet_2023-07-20' has already been downloaded.
INFO:keyedvectors.py:2047: loading projection weights from ../data/lemmas.cbow.s100.w2v.bin
INFO:utils.py:448: KeyedVectors lifecycle event {'msg': 'loaded (441391, 100) matrix of type float32 from ../data/lemmas.cbow.s100.w2v.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-05-07T19:27:24.690203', 'gensim': '4.3.2', 'python': '3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


In [ ]:
from crossword_solver.crossword_solver import solve_crossword

In [16]:
path = '../data/pictures/ristsona_2.png'
crossword = detect_crossword_from_file(path)

In [17]:
reorder_crossword_hints(crossword)

In [18]:
find_whole_crossword_candidates(crossword)

In [19]:
len(crossword.hints)

41

In [ ]:
min_score = 5
results = list()
for matrix, score in solving_algorithm(crossword, max_empty_words=0):
    if score > min_score:
        results.append((matrix, score))

In [ ]:
path = '../data/pictures/ristsona_1.png'
results = solve_crossword(path)

In [ ]:
from collections import defaultdict
import time
timings = defaultdict(float)
min_score = 5
results = list()
for matrix, score in solving_algorithm(crossword, max_empty_words=0):
    if score > min_score:
        results.append((matrix, score))

print(len(results))
timings

In [ ]:
find_whole_crossword_candidates(crossword)
print(len(crossword.hints))

In [ ]:
reorder_crossword_hints(crossword)
print(len(crossword.candidates))

In [ ]:
crossword

In [ ]:
for hint in crossword.hints:
    print(hint.hint, hint.length, len(hint.candidates))

In [ ]:
import numpy as np
from crossword_solver.solver_helpers import find_suitable_candidates
def solving_algorithm(crossword, max_empty_words = 5):
    global timings
    if len(crossword.hints)==0:
        yield np.copy(crossword.matrix), crossword.score
        return
  
    start = time.time()
    hint = crossword.hints.pop(0)
    suitable_candidates = find_suitable_candidates(hint, crossword)
    timings["find_suitable_candidates"] += time.time()-start
    if len(suitable_candidates)==0 and max_empty_words==0:
        yield np.copy(crossword.matrix), crossword.score
    if max_empty_words>0:
        yield from solving_algorithm(crossword, max_empty_words-1)
    for candidate in suitable_candidates:
        start = time.time()
        prev_text = np.copy(crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1])
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = np.array(list(candidate.text)
                                                                                          ).reshape(prev_text.shape)
        crossword.score += candidate.weight
        timings["crossword_manip"] += time.time()-start
        yield from solving_algorithm(crossword, max_empty_words)
        start = time.time()
        crossword.score -= candidate.weight
        crossword.matrix[hint.x_min:hint.x_max+1, hint.y_min:hint.y_max+1] = prev_text
        timings["crossword_manip"] += time.time()-start
    crossword.hints.insert(0, hint)

In [ ]:
printing_crossword = deepcopy(crossword)
for matrix, score in sorted(results, key=lambda x: x[1], reverse = True)[:5]:
    printing_crossword.matrix = matrix
    print(printing_crossword, score)